In [143]:
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime


In [144]:
#List of fields which we are interested from the dataframe

criteria_list = ['movie_id', 'title', 'review_count_user', 'review_count_critic','rating_value', 'rating_count','date_published', 'director_names', 'director_ids']
dup_criteria_list = ['movie_id', 'title', 'review_count_user', 'review_count_critic','rating_value', 'rating_count','date_published']



In [145]:
# Creating a dummy dataframe with the required column fields

sampleSkeleton = pd.DataFrame(columns=criteria_list)

# Directory path to the folder containing IMBD .dat files

#workingDir = "C:\\Yuva\\ITU\\3rd Sem\\Research Topics\\Movies & Gender\\Data\\IMDb\\Sample"
workingDir = "C:\\Yuva\\ITU\\3rd Sem\\Research Topics\\Movies & Gender\\Data\\IMDb\\movie_info\\movie_bios"

# List of file names in the working directory, i.e. 700000+ records

allFiles = os.listdir(workingDir)

print(workingDir+"\\"+allFiles[0])

C:\Yuva\ITU\3rd Sem\Research Topics\Movies & Gender\Data\IMDb\movie_info\movie_bios\added


In [146]:
# Empty errorlist

errorList=["None"]
countFiles = 1

In [147]:
start = time.time()
print("Started Reading files at :" + str(datetime.utcfromtimestamp(start).strftime('%Y-%m-%d %H:%M:%S')))

# Loop through AllFiles in the directory.

for file in allFiles:
# Read only the files with .dat extension
    if file.endswith('.dat'):
         if countFiles<50000:                
# Read the file using pandas as it is efficient and takes care of the memory management.
            filePath = pd.read_fwf(workingDir+"\\"+file, header=None, sep="\t",nrows=20)
    # Store the data
            allTempData = filePath[0].str.split("\t", n = 1, expand = True)
            # Check if the file we read is in the desired format
            if(len(allTempData.columns)==2):
                try:
                    reqData = allTempData.set_index(0).T[criteria_list]
                    # Expand = true is for the splitting to return not as lists
                    reqData['director_names'] = reqData['director_names'].str.split("\t", expand = False)
                    reqData.director_ids = reqData.director_ids.str.split("\t", expand = False)
                    if(len(reqData['director_names'][1])==1):
                        sampleSkeleton = sampleSkeleton.append([reqData],ignore_index=True)
                    else:
                        # To keep a count of directors per movie
                        loop = len(reqData['director_names'][1])
                        # Make duplicate rows to the number of values in the director names column , i.e loop count -1 
                        reqData = reqData.append([reqData]*(loop-1),ignore_index=True)
                        for i in range(loop):
                            # Leading spaces from the source files. Hence lstrip(' ')
                            # The below replaces the rows with the corresponding director name and director ID
                            reqData.director_names[i] = [reqData.director_names[i][i].lstrip(' ')]
                            reqData.director_ids[i] = [reqData.director_ids[i][i].lstrip(' ')]
                        sampleSkeleton = sampleSkeleton.append(reqData,ignore_index=True)
                    countFiles += 1
                except:
                    errorList.append("File Name : " + file + " , Number of columns:" + str(len(reqData.columns)))

            else:
                errorList.append("File Name : " + file + " , Number of columns: Incorrect format" )

# Create a CSV and save the file information. Index FALSE removes the row indices
sampleSkeleton.to_csv(workingDir + "\\Final.csv",index= False)

# Stop timer
end = time.time()
print("Stopped Reading files at :" + str(datetime.utcfromtimestamp(end).strftime('%Y-%m-%d %H:%M:%S')))
print("Total time taken in minutes: " + str((end - start)/60))
#print(sampleSkeleton)
#sampleSkeleton


        
    

Started Reading files at :2019-10-10 06:26:29
Stopped Reading files at :2019-10-10 07:50:37
Total time taken in minutes: 84.13125938177109


In [148]:
# Write the error List to a file

with open(workingDir + '\\errorFinal.txt', 'w') as f:
    for item in errorList:
        f.write("%s\n" % item)

In [ ]:
directorid = sampleSkeleton.director_ids.apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index', 'value']].set_index('index')
directorid

In [521]:
directorNames = sampleSkeleton.director_names.apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index', 'value']].set_index('index')
directorNames

,value
index,
0,William K.L. Dickson
1,Ã‰mile Reynaud
2,Ã‰mile Reynaud
3,Ã‰mile Reynaud
4,William K.L. Dickson (uncredited)
5,William K.L. Dickson (uncredited)
6,William K.L. Dickson
7,William K.L. Dickson
8,Alexander Black


In [553]:
dupdf = pd.merge(sampleSkeleton[dup_criteria_list],
    pd.merge(
        directorNames,
        directorid,
        left_index=True,
        right_index=True),
    left_index=True,
    right_index=True).rename(columns={'value_x': 'director_names', 'value_y': 'director_id'})

dupdf

#sortDF = dupdf.sort_values(by=['director_names','director_id'],ascending = [0,1])
dropdupDF = dupdf.drop_duplicates(subset = ['title','director_names'],keep = 'first')



In [551]:
dupdf

,movie_id,title,review_count_user,review_count_critic,rating_value,rating_count,date_published,director_names,director_id
0,tt0000001,Carmencita - spanyol tÃ¡nc (1894) - IMDb,12,2,5.8,1255,1894-03-10,William K.L. Dickson,nm0005690
1,tt0000002,A bohÃ³c Ã©s kutyÃ¡i (1892) - IMDb,0,None,6.5,148,1892-10-28,Ã‰mile Reynaud,nm0721526
2,tt0000003,SzegÃ©ny Pierrot (1892) - IMDb,12,5,6.6,819,1892-10-28,Ã‰mile Reynaud,nm0721526
3,tt0000004,Egy jÃ³ pohÃ¡r sÃ¶r (1892) - IMDb,1,None,6.6,90,1892-10-28,Ã‰mile Reynaud,nm0721526
4,tt0000005,A patkolÃ³kovÃ¡cs (1893) - IMDb,18,3,6.2,1499,1893-05-09,William K.L. Dickson (uncredited),nm0005690
5,tt0000006,KÃ­nai Ã³piumbarlang (1894) - IMDb,None,None,5.7,70,1894-10-17,William K.L. Dickson (uncredited),nm0005690
6,tt0000007,A Corbett-Courtney bokszmeccs (1894) - IMDb,5,2,5.5,499,2008-12-31,William K.L. Dickson,nm0005690
6,tt0000007,A Corbett-Courtney bokszmeccs (1894) - IMDb,5,2,5.5,499,2008-12-31,William K.L. Dickson,nm0374658
6,tt0000007,A Corbett-Courtney bokszmeccs (1894) - IMDb,5,2,5.5,499,2008-12-31,William Heise,nm0005690
6,tt0000007,A Corbett-Courtney bokszmeccs (1894) - IMDb,5,2,5.5,499,2008-12-31,William Heise,nm0374658


In [554]:
dropdupDF

,movie_id,title,review_count_user,review_count_critic,rating_value,rating_count,date_published,director_names,director_id
0,tt0000001,Carmencita - spanyol tÃ¡nc (1894) - IMDb,12,2,5.8,1255,1894-03-10,William K.L. Dickson,nm0005690
1,tt0000002,A bohÃ³c Ã©s kutyÃ¡i (1892) - IMDb,0,None,6.5,148,1892-10-28,Ã‰mile Reynaud,nm0721526
2,tt0000003,SzegÃ©ny Pierrot (1892) - IMDb,12,5,6.6,819,1892-10-28,Ã‰mile Reynaud,nm0721526
3,tt0000004,Egy jÃ³ pohÃ¡r sÃ¶r (1892) - IMDb,1,None,6.6,90,1892-10-28,Ã‰mile Reynaud,nm0721526
4,tt0000005,A patkolÃ³kovÃ¡cs (1893) - IMDb,18,3,6.2,1499,1893-05-09,William K.L. Dickson (uncredited),nm0005690
5,tt0000006,KÃ­nai Ã³piumbarlang (1894) - IMDb,None,None,5.7,70,1894-10-17,William K.L. Dickson (uncredited),nm0005690
6,tt0000007,A Corbett-Courtney bokszmeccs (1894) - IMDb,5,2,5.5,499,2008-12-31,William K.L. Dickson,nm0005690
6,tt0000007,A Corbett-Courtney bokszmeccs (1894) - IMDb,5,2,5.5,499,2008-12-31,William Heise,nm0005690
7,tt0000008,Edison asszisztense tÃ¼sszent (1894) - IMDb,19,4,5.6,1341,1894-01-09,William K.L. Dickson,nm0005690
8,tt0000009,Miss Jerry (1894) - IMDb,1,2,5.6,58,1894-10-09,Alexander Black,nm0085156


# Trying to split

Into tabs

In [ ]:
# If required, to further split the second column in the dataframe to have tokenize 
# data[1] = data[1].str.split("\t", expand = False)

In [ ]:
#data[1][15][2]